# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor


### import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Any results you write to the current directory are saved as output.

import re
import datetime
from datetime import datetime, timedelta

In [2]:
# проверка на работу в "google colab" или "локально"

if 'sample_data' in os.listdir():
    project_dir = r'/content/drive/My Drive/Colab Notebooks/module_3/'
    print('Обнаружена среда выполнения Google Colab.')
    print('project_dir =>', project_dir)
else:
    project_dir = ''
    print('НЕ обнаружена среда выполнения Google Colab. Выбран режим локальной работы.')


НЕ обнаружена среда выполнения Google Colab. Выбран режим локальной работы.


In [3]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

### Функции обработки


In [4]:
def reviews_word_analyzer(incoming, mode='pos'):
    ''' Оценка позитивной\негативной окраски текстового сообщения.
        Проверяется каждое слово на вхождение в список позитивных или негативных словарей.
        Возвращается количество найденных вхождений.
    '''
    word_count = 0
    # определяем справочник для проверки вхождения слов   
    if type(incoming) is list:
        incoming = ','.join(incoming)
    if mode == 'pos':
        control_list = pos_words
    else:
        control_list = neg_words
    # подсчитываем количество вхождений    
    for word in re.findall(r'[a-zA-Z\-]+', incoming):
        if word in control_list:
            word_count += 1
    return word_count
# end function


In [5]:
def column_content_analysis(column_name_list, column_type='str', more_than = 10):
    '''
        Функция для определения типовых параметров данных, а также определние "замусоренности".
        На вход принимает список столбцов датафрейма
    '''
 # проходим циклом по списку колонок    
    for column_name in column_name_list:
        print('-'*60)
        print('|\t' , 'Отчет по колонке [', column_name, ']', '\t'*5)
        print('-'*60)
        row_count = data.shape[0]
        if column_name not in data.columns:
            print('|  Внимание!!! Указанная колонка не найдена в датасете. \n')
            break

        col_count = data[column_name].count()
        col_type = data[column_name].dtype
        print('|  Тип данных:', col_type) 
        print('|  Заполнено значений:',  col_count, 'из', row_count)
        print('|  Отсутсвующие значения:', row_count - col_count)
        print('|  Полнота данных: ', round(col_count / row_count * 100 ,2), '%', sep='')
        print('|  Количество уникальных значений:', data[column_name].nunique())

        print('|  Значений, встретившихся в столбце более', more_than,'раз:', 
              (data[column_name].value_counts() > more_than).sum())

        if col_type == 'object':
            # Количество пробелов в начале в конце строки
            print('|  Количество раз пробелы найдены:')
            find_list = [len(re.findall(r'^\s+\w?', str(x))) for x in data[column_name]]
            print('|    в начале строки:', sum(find_list))
            
            find_list = [len(re.findall(r'\w?\s+$', str(x))) for x in data[column_name]]
            print('|    в конце  строки:', sum(find_list))
            
            find_list = [len(re.findall(r'[^\s\d\w]', str(x))) for x in data[column_name]]
            print('|  Количество найденных специальных символов:', sum(find_list))
        
        if col_type in ['int', 'float']:
            print('|  Медиана: ', data[column_name].median())
            
            perc25 = data[column_name].describe().loc['25%']
            perc75 = data[column_name].describe().loc['75%']
            iqr = perc75 - perc25
            range_left = perc25 - 1.5 * iqr
            range_right = perc75 + 1.5 * iqr
            
            # Границы выбросов
            print('|  IQR: ', iqr, '; границы выбросов: [', round(range_left, 2), '; ', round(range_right,2),']' , sep='')
            out_of_range_left = data[column_name].loc[data[column_name] < range_left]
            out_of_range_right = data[column_name].loc[data[column_name] > range_right]
            out_of_range = len(out_of_range_left) + len(out_of_range_right)
            if out_of_range > 0:
                print('|  Данные СОДЕРЖАТ выбросы, количество значений: ', out_of_range)
                if len(out_of_range_left) > 0:
                    print('|    Слева  от IQR (уникальные):', out_of_range_left.unique())
                if len(out_of_range_right) > 0:
                    print('|    Справа от IQR (уникальные):', out_of_range_right.unique())
         
        print('|---\n|  Все уникальные значения:', data[column_name].unique() , '\n')


    # end function


# DATA

In [6]:
# Выведем листинг проектного каталога
for dirname, _, filenames in os.walk(project_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [7]:

df_train = pd.read_csv(project_dir + 'main_task.csv')
df_test = pd.read_csv(project_dir + 'kaggle_task.csv')
sample_submission = pd.read_csv(project_dir + 'sample_submission.csv')


FileNotFoundError: [Errno 2] File main_task.csv does not exist: 'main_task.csv'

In [ ]:
# подключаем дополнительные источники - словари

negotive = pd.read_csv(project_dir + 'neg_words.txt', names=['neg_word'], encoding='latin-1')
positive = pd.read_csv(project_dir + 'positive-words.txt', names=['pos_word'], encoding='latin-1', comment=';')

negotive.shape, positive.shape

In [ ]:
# словарь негативно окрашенных слов
negotive.sample(9)

In [ ]:
# подготовим список негативных слов
neg_words = negotive['neg_word'].to_list()

In [ ]:
# словарь позитивно окрашенных слов
positive.sample(9)

In [ ]:
# подготовим список слов с позитивной окраской
pos_words = positive['pos_word'].to_list()

In [ ]:
df_train.info()

In [ ]:
df_train.sample(5)

In [ ]:
df_test.info()

In [ ]:
df_test.sample(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

## Информация о колонках

In [ ]:
# выведем детальный отчет по анализу данных в колонках

column_content_analysis(data.columns, more_than=100)

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

## Cleaning and Prepping Data


## 1. Обработка NAN 


In [ ]:
# возмем столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [ ]:
data['Number_of_Reviews_isNAN']

In [ ]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Number of Reviews'].fillna(0, inplace=True)


# 2. Обработка признаков
Посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

Указанные признаки можно считать категориальными
* City
* Cuisine Style



In [ ]:
# посмотрим на один из отзывов
data.Reviews[1]

Потребуется парсинг и разборе признака. Текстовые отзывы можно обработать на факт эмоционального окраса отзыва. После обрабокти сформировать дополнительные признаки.

Даты отзывов можно трансформировать в признак, посчитав длительность между ними, например в минутах.

## Кодирование признаков

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:


In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [ ]:
data.sample(5)

## "Price Range"

In [ ]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  


In [ ]:
# обработка 'Price Range'

# подготовим словарь для конвертации значений
price_range_dict = dict(zip(['$$ - $$$', '$', '$$$$'], [500, 5, 5000]))

data['Price Range'] = data['Price Range'].replace(to_replace=price_range_dict)

In [ ]:
data['Price Range'].value_counts()

In [ ]:
data.columns[:11]

## "Cuisine Style"

In [ ]:
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: re.findall(r"[^'][\w\s]+[^']", str(x)))


In [ ]:
%%time
data = pd.get_dummies(data.explode('Cuisine Style'), columns=['Cuisine Style'], prefix='CS', dummy_na=True)


# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.


### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.



### Распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### Корреляция признаков
На этом графике можно заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

# Data Preprocessing
Для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(project_dir + 'main_task.csv')
df_test = pd.read_csv(project_dir + 'kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA'], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски
    df_output['Number of Reviews'].fillna(0, inplace=True)
    # тут ваш код по обработке NAN
    df_output['Price Range'].fillna('0', inplace=True)
    
    # для нулевых значений поставим средние значения по городу
    number_of_reviews_mean = df_output.groupby('City')['Number of Reviews'].mean()
    df_output['Number of Reviews'] = df_output.apply(lambda row: number_of_reviews_mean[row['City']] if row['Number of Reviews'] == 0 else row['Number of Reviews'], axis=1)
    
    
    # ################### 3. Encoding ############################################################## 
    # get_dummies
    
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    
    price_range_dict = dict(zip(['$$ - $$$', '$', '$$$$', '0'], [500, 5, 5000, 0]))
    df_output['Price Range'] = df_output['Price Range'].replace(to_replace=price_range_dict)

    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(lambda x: re.findall(r"[^'][\w\s]+[^']", str(x)))

    df_output = pd.concat([df_output, 
                 pd.get_dummies(df_output.explode('Cuisine Style')['Cuisine Style'], dummy_na=True)], axis = 1)
    
    
    
    # ################### 4. Feature Engineering ####################################################
    #

    df_output['score_pos'] = df_output['Reviews'].apply(lambda x: reviews_word_analyzer(str(x), mode='pos'))

    df_output['score_neg'] = df_output['Reviews'].apply(lambda x: reviews_word_analyzer(str(x), mode='neg'))
    
    
    df_output['reviews_data'] = df_output['Reviews'].apply(lambda x: re.findall(r"\d\d\/\d\d\/\d{4}", str(x)) 
                                                           if len(re.findall(r"\d\d\/\d\d\/\d{4}", str(x))) == 2
                                                           else ['01/01/2000', '01/01/2000'])

    df_output['reviews_delta'] = df_output['reviews_data'].apply(lambda x: (datetime.timestamp(datetime.strptime(x[0], '%m/%d/%Y')) 
                                                                    - datetime.timestamp(datetime.strptime(x[1], '%m/%d/%Y')))/60 )
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберем их и удалим
    
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output


Новые признаки:

1.  `score_pos` - числовая оценка положительного окраса отзыва
2.  `score_neg` - числовая оценка отрицтельного окраса отзыва
3. `reviews_delta` - количество минут между послденим и предпоследним отзывом



#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.info()

In [ ]:
df_preproc.sample(9)

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

### Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
%%time
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

In [ ]:
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

# MAE

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))


*  Исходное значение метрики:
> MAE: 0.21240125

* Добавлен обработчик пустых значений для количества отзывов. Устанавливает среднее по городу.  Изменение метрики:
> MAE: 0.0995425

* Добавлена дельта времени между отзывами в минутах.  Изменение метрики:
> MAE: 0.089500

*  Небольшое изменение логики обработки пустых значений.  Изменение метрики:
> MAE: 0.089434

* Добавлена оценка отзывов на наличие негативных или позитивных слов. Изменение метрики:
> MAE: 0.0875



In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

### Submission
Готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
# приведем рейтинг к кратности шага 0.5
predict_submission = list(map(lambda x: round(x * 2)/2, predict_submission))

In [ ]:
# посмотрим что получилось
len(predict_submission), predict_submission[:10]

In [ ]:
sample_submission['Rating'] = predict_submission[0:10000]
sample_submission.to_csv(project_dir + 'submission.csv', index=False)
sample_submission.head(10)

# Выводы

Чтобы улучшить результат:
* Обработали и превратили оставшиеся признаки в понятный для машины формат - цифровые признаки.
* Придумали, что еще можно извлечь из признаков и расширили набор исходных данных.
* Сгенерировать новые признаки.
* Подгрузить дополнительные данные - обработка содержрания отзывов на ключевые показатели.
* Подобрали состав признаков.
* Структурное изменение датафрейма с `11` до `167` столбцов.
* Сократили `MAE`: c `0.2124` до `0.0875`


# Спасибо за внимание!